In [1]:
from pyspark import SparkContext, SparkConf # 导入相关工具包

# 初始化Spark上下文
# local为本地调试模式，具体集群方式参照http://spark.apache.org/docs/latest/cluster-overview.html
conf = SparkConf().setAppName("CF").setMaster("local") 
sc = SparkContext(conf=conf)
print ("init complete：sc = ", sc)

('init complete\xef\xbc\x9asc = ', <SparkContext master=local appName=CF>)


In [11]:
# 导入数据，数据格式为：user_id::movies_id::rating::time
fileData = sc.textFile("file:///root/notebook/data/ratings.dat").map(lambda line: line.split("::"))
print("data.count() = %d" % fileData.count())

# 数据预处理，根据评论时间戳最后一位把整个数据集分成训练集(60%), 交叉验证集(20%), 和评估集(20%)
data = fileData.map(lambda fields: (long(fields[3]) % 10, (int(fields[0]), int(fields[1]), float(fields[2]))))
training = data.filter(lambda x: x[0] < 6).values()
validation = data.filter(lambda x: x[0] >= 6 and x[0] < 8).values()
test = data.filter(lambda x: x[0] >= 8).values()

print("training.count() = %d,validation.count() = %d,test.count() = %d" % (
    training.count(), validation.count(), test.count()))

data.count() = 10000
training.count()=5932,validation.count()=2001,test.count()=2067


In [14]:
ranks = [8, 12]
lambdas = [1.0, 10.0]
numIters = [10, 20]
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

import itertools
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    print(rank, lmbda, numIter)

(8, 1.0, 10)
(8, 1.0, 20)
(8, 10.0, 10)
(8, 10.0, 20)
(12, 1.0, 10)
(12, 1.0, 20)
(12, 10.0, 10)
(12, 10.0, 20)
